In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\Asus\\vs_code\\Internship\\News_web_scraping\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus\\vs_code\\Internship\\News_web_scraping'

In [5]:
from News_web_scraping import logger 
from News_web_scraping.components.Model import ModelSetup
from News_web_scraping.utils.common import * 
from News_web_scraping.config.configuration import ConfigurationManager
from News_web_scraping.entity.config_entity import ModelConfig


from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [6]:
manager             = ConfigurationManager()
model_config        = manager.get_model_config()

[2024-10-04 15:46:54,685: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-04 15:46:54,689: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-04 15:46:54,689: INFO: common: created directory at: artifacts]
[2024-10-04 15:46:54,691: INFO: configuration: Model config initialized]


In [7]:
class PromptAndChain(ModelSetup):

    def __init__(self,model_config: ModelConfig,):
        
        super().__init__(model_config)
        self.llm                        = self.model_setup()
        self.parser                     = JsonOutputParser()

    def prompt_and_chain(self): 

        logger.info(f"Prompting and chain has started")

        json_prompt = """
You are an expert information extractor. Your task is to extract relevant data from the following text and organize it into a structured JSON format.

Instructions:
    1. Carefully read the following text.
    2. Extract key information and format it into a JSON object.
    3. Ensure that the keys in the JSON are relevant to the content. Use meaningful key names based on the context of the text.
    4. **Strict formatting rules**:
        - The output must be a valid JSON object.
        - If a value is a string, enclose it in double quotes.
        - If a value is a number, boolean, or another type, do not enclose it in quotes.

Text:
{document_text}

Output:
Provide a JSON object containing the extracted relevant information.Ensure that the final output adheres strictly to this format.
"""

        example_prompt  = PromptTemplate(
            input_variables = ["document_text"],
            template        = json_prompt)

        chain = (
            {
                "document_text" : RunnablePassthrough() 
            }
            | example_prompt 
            | self.llm
            | self.parser
        )
        return chain 


In [8]:

if __name__ == "__main__":
    manager             = ConfigurationManager()
    model_config        = manager.get_model_config()
    prompt_and_chain    = PromptAndChain(model_config=model_config)
    chain               = prompt_and_chain.prompt_and_chain()
    

[2024-10-04 15:46:54,720: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-04 15:46:54,722: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-04 15:46:54,722: INFO: common: created directory at: artifacts]
[2024-10-04 15:46:54,722: INFO: configuration: Model config initialized]
[2024-10-04 15:46:54,724: INFO: Model: Model setup initialized]
[2024-10-04 15:46:55,213: INFO: Model: model----llama3-8b-8192----created]
[2024-10-04 15:46:55,217: INFO: 1713379584: Prompting and chain has started]


In [10]:
response = chain.invoke("hey my name is arjun, and im working as an engineer")

[2024-10-04 16:46:48,054: INFO: _client: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"]


In [11]:
json.dumps(response)

'{"Name": "arjun", "Occupation": "engineer"}'